In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the dataset
dataset = pd.read_csv('../data/dataset.csv')


# Display basic info of the dataset to check for missing values and data types
print("Dataset Information:")
print(dataset.info())
print("Dataset Description:")
print(dataset.describe())

# Step 1: Feature Engineering (optional, you can add more interactions based on domain knowledge)
# Example: Creating an interaction feature between mindset and school achievement level
dataset['Mindset_School_Achievement_Interaction'] = dataset['X1'] * dataset['X2']

# Step 2: Define covariates (features), treatment, and outcome
outcome_col = 'Y'  # Student Achievement Score (Outcome)
treatment_col = 'Z'  # Growth Mindset Intervention (Treatment)
covariate_cols = ['S3', 'C1', 'C2', 'C3', 'XC', 'X1', 'X2', 'X3', 'X4', 'X5', 'Mindset_School_Achievement_Interaction']

# Step 3: Categorical and Numerical Feature Identification
# Categorical columns that need one-hot encoding
categorical_cols = ['C1', 'C2', 'C3', 'XC']

# Numerical columns that need to be standardized
numerical_cols = ['X1', 'X2', 'X3', 'X4', 'X5', 'Mindset_School_Achievement_Interaction']

# Step 4: Preprocessing Pipeline
# A pipeline that will scale numerical features and one-hot encode categorical ones
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),  # Scaling numerical columns
        ('cat', OneHotEncoder(drop='first'), categorical_cols)  # One-hot encoding categorical columns
    ])

# Step 5: Train-Test Split
X = dataset[covariate_cols]  # Features (covariates)
y = dataset[outcome_col]  # Outcome (Student Achievement Score)

# Split data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Apply the Preprocessing Pipeline
# Fit the preprocessor on the training data and transform both train and test sets
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Retrieve the feature names after one-hot encoding and scaling
# This ensures that our transformed data has the correct column names
feature_names = numerical_cols + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols))

# Convert the transformed data back to DataFrames for easier use
X_train_df = pd.DataFrame(X_train_transformed, columns=feature_names)
X_test_df = pd.DataFrame(X_test_transformed, columns=feature_names)

# Step 7: Output the results to verify the transformations
print("Transformed Training Set:")
print(X_train_df.head())
print("\nTransformed Test Set:")
print(X_test_df.head())

# Output the shapes of the datasets
print(f"\nTraining data shape: {X_train_df.shape}, Test data shape: {X_test_df.shape}")


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10391 entries, 0 to 10390
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   schoolid  10391 non-null  int64  
 1   Z         10391 non-null  int64  
 2   Y         10391 non-null  float64
 3   S3        10391 non-null  int64  
 4   C1        10391 non-null  int64  
 5   C2        10391 non-null  int64  
 6   C3        10391 non-null  int64  
 7   XC        10391 non-null  int64  
 8   X1        10391 non-null  float64
 9   X2        10391 non-null  float64
 10  X3        10391 non-null  float64
 11  X4        10391 non-null  float64
 12  X5        10391 non-null  float64
dtypes: float64(6), int64(7)
memory usage: 1.0 MB
None
Dataset Description:
           schoolid             Z             Y            S3            C1  \
count  10391.000000  10391.000000  10391.000000  10391.000000  10391.000000   
mean      39.888846      0.325666     -0.096742    

In [2]:
# Import necessary libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import numpy as np

# Function to handle missing values and train the S-Learner
def s_learner(X_train, X_test, y_train, y_test, treatment_train, treatment_test,s_model):
    # Append treatment column to training and test sets
    X_train_with_treatment = X_train.copy()
    X_test_with_treatment = X_test.copy()
    
    # Add treatment indicator to the feature set
    X_train_with_treatment['treatment'] = treatment_train
    X_test_with_treatment['treatment'] = treatment_test
    
    # Handle missing values by imputing them
    imputer = SimpleImputer(strategy='mean')  # For numerical values
    X_train_imputed = imputer.fit_transform(X_train_with_treatment)
    X_test_imputed = imputer.transform(X_test_with_treatment)
    
    # Use a RandomForestRegressor as the base model for the S-learner
    #s_model = RandomForestRegressor(n_estimators=100, random_state=42)
    
    # Train the model on the combined feature set (covariates + treatment indicator)
    s_model.fit(X_train_imputed, y_train)
    
    # Predict outcomes for both treated and untreated cases
    X_test_with_treatment['treatment'] = 1  # Treated case
    y_pred_treated = s_model.predict(X_test_imputed)
    
    X_test_with_treatment['treatment'] = 0  # Untreated case
    y_pred_control = s_model.predict(X_test_imputed)
    
    # Calculate the treatment effect (difference between treated and control)
    treatment_effect = y_pred_treated - y_pred_control
    
    # Evaluate the model performance using mean squared error (MSE)
    mse = mean_squared_error(y_test, s_model.predict(X_test_imputed))
    print(f"S-Learner MSE: {mse}")
    
    return treatment_effect, s_model

# Extract treatment column (Z) for both training and testing datasets
treatment_train = dataset.loc[X_train.index, 'Z']
treatment_test = dataset.loc[X_test.index, 'Z']

 # Use a RandomForestRegressor as the base model for the S-learner
s_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train and evaluate the S-learner with missing value handling
s_treatment_effect, s_model = s_learner(X_train_df, X_test_df, y_train, y_test, treatment_train, treatment_test,s_model)

# Print the estimated treatment effects
print("Estimated treatment effects (S-Learner):\n", s_treatment_effect[:5])  # Showing first 5 treatment effects
print("Treatment Distribution in Training Set:")
print(treatment_train.value_counts())

S-Learner MSE: 0.45278502287687317
Estimated treatment effects (S-Learner):
 [0. 0. 0. 0. 0.]
Treatment Distribution in Training Set:
Z
0    5601
1    2711
Name: count, dtype: int64


In [3]:
# Import necessary libraries
from sklearn.utils import resample

# Combine training data with treatment indicator and target
X_train_with_treatment = X_train_df.copy()
X_train_with_treatment['Z'] = treatment_train
X_train_with_treatment['Y'] = y_train  # Add the target to the dataset

# Split treated and untreated groups
treated = X_train_with_treatment[X_train_with_treatment['Z'] == 1]
untreated = X_train_with_treatment[X_train_with_treatment['Z'] == 0]

# Oversample treated group to match the size of the untreated group
treated_oversampled = resample(treated, 
                               replace=True,  # Sample with replacement
                               n_samples=len(untreated),  # Match untreated sample size
                               random_state=42)

# Combine the oversampled treated group with the untreated group
balanced_train = pd.concat([untreated, treated_oversampled])

# Separate the features, treatment labels, and target again
X_train_balanced = balanced_train.drop(columns=['Z', 'Y'])  # Features
treatment_train_balanced = balanced_train['Z']  # Treatment indicator
y_train_balanced = balanced_train['Y']  # Target (Outcome)

 # Use a RandomForestRegressor as the base model for the S-learner
s_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Now train the S-learner on the balanced dataset
s_treatment_effect_balanced, s_model_balanced = s_learner(X_train_balanced, X_test_df, y_train_balanced, y_test, treatment_train_balanced, treatment_test,s_model)

# Print the estimated treatment effects
print("Estimated treatment effects (S-Learner, Balanced):\n", s_treatment_effect_balanced[:5])


S-Learner MSE: 0.520193741079241
Estimated treatment effects (S-Learner, Balanced):
 [0. 0. 0. 0. 0.]


In [4]:
# Import necessary libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Function to train and evaluate a T-Learner with aligned indices
def t_learner(X_train, X_test, y_train, y_test, treatment_train, treatment_test):
    # Reset indices to ensure alignment
    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    treatment_train = treatment_train.reset_index(drop=True)
    
    # Split the training set into treated and control groups
    X_train_treated = X_train[treatment_train == 1]
    y_train_treated = y_train[treatment_train == 1]
    
    X_train_control = X_train[treatment_train == 0]
    y_train_control = y_train[treatment_train == 0]
    
    # Use a RandomForestRegressor as the base model for both treated and control groups
    treated_model = RandomForestRegressor(n_estimators=100, random_state=42)
    control_model = RandomForestRegressor(n_estimators=100, random_state=42)
    
    # Train models separately for treated and control groups
    treated_model.fit(X_train_treated, y_train_treated)
    control_model.fit(X_train_control, y_train_control)
    
    # Predict outcomes for the test set using both models
    y_pred_treated = treated_model.predict(X_test)  # Treated model predictions
    y_pred_control = control_model.predict(X_test)  # Control model predictions
    
    # Calculate the treatment effect (difference between treated and control predictions)
    treatment_effect = y_pred_treated - y_pred_control
    
    # Evaluate the models using MSE
    mse_treated = mean_squared_error(y_test, y_pred_treated)
    mse_control = mean_squared_error(y_test, y_pred_control)
    
    print(f"Treated Model MSE: {mse_treated}")
    print(f"Control Model MSE: {mse_control}")
    
    return treatment_effect, treated_model, control_model

# Ensure the treatment column for training and test sets has matching indices with features
treatment_train_aligned = treatment_train.reset_index(drop=True)
treatment_test_aligned = treatment_test.reset_index(drop=True)

# Train and evaluate the T-Learner
t_treatment_effect, treated_model, control_model = t_learner(X_train_df, X_test_df, y_train, y_test, treatment_train_aligned, treatment_test_aligned)

# Print the estimated treatment effects
print("Estimated treatment effects (T-Learner):\n", t_treatment_effect[:5])  # Showing first 5 treatment effects


Treated Model MSE: 0.47066167057112024
Control Model MSE: 0.44565126147139134
Estimated treatment effects (T-Learner):
 [0.11595774 0.43627006 0.13755668 1.16536816 0.6950525 ]


In [5]:
# Import necessary libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Function to train and evaluate an X-Learner with MSE evaluation
def x_learner(X_train, X_test, y_train, y_test, treatment_train, treatment_test):
    # Step 1: Train T-Learner models (treated and control)
    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    treatment_train = treatment_train.reset_index(drop=True)
    
    X_train_treated = X_train[treatment_train == 1]
    y_train_treated = y_train[treatment_train == 1]
    
    X_train_control = X_train[treatment_train == 0]
    y_train_control = y_train[treatment_train == 0]
    
    treated_model = RandomForestRegressor(n_estimators=100, random_state=42)
    control_model = RandomForestRegressor(n_estimators=100, random_state=42)
    
    treated_model.fit(X_train_treated, y_train_treated)
    control_model.fit(X_train_control, y_train_control)
    
    # Step 1 MSE: Evaluate T-Learner models
    y_pred_treated = treated_model.predict(X_test)
    y_pred_control = control_model.predict(X_test)
    mse_treated = mean_squared_error(y_test, y_pred_treated)
    mse_control = mean_squared_error(y_test, y_pred_control)
    print(f"MSE for Treated Model (T-Learner): {mse_treated}")
    print(f"MSE for Control Model (T-Learner): {mse_control}")
    
    # Step 2: Impute outcomes for the opposite group
    y_imputed_control_for_treated = control_model.predict(X_train_treated)  # Impute control outcomes for treated group
    y_imputed_treated_for_control = treated_model.predict(X_train_control)  # Impute treated outcomes for control group
    
    # Step 3: Calculate imputed treatment effects (pseudo-outcomes)
    pseudo_outcome_treated = y_train_treated - y_imputed_control_for_treated
    pseudo_outcome_control = y_imputed_treated_for_control - y_train_control
    
    # Step 4: Train final models on pseudo-outcomes
    final_treated_model = RandomForestRegressor(n_estimators=100, random_state=42)
    final_control_model = RandomForestRegressor(n_estimators=100, random_state=42)
    
    final_treated_model.fit(X_train_treated, pseudo_outcome_treated)
    final_control_model.fit(X_train_control, pseudo_outcome_control)
    
    # Step 4 MSE: Evaluate pseudo-outcome models
    pseudo_pred_treated = final_treated_model.predict(X_train_treated)
    pseudo_pred_control = final_control_model.predict(X_train_control)
    mse_pseudo_treated = mean_squared_error(pseudo_outcome_treated, pseudo_pred_treated)
    mse_pseudo_control = mean_squared_error(pseudo_outcome_control, pseudo_pred_control)
    print(f"MSE for Pseudo-Outcome Model (Treated): {mse_pseudo_treated}")
    print(f"MSE for Pseudo-Outcome Model (Control): {mse_pseudo_control}")
    
    # Step 5: Predict treatment effects for the test set
    treatment_effect_treated = final_treated_model.predict(X_test)  # Final model predictions for treated pseudo-outcome
    treatment_effect_control = final_control_model.predict(X_test)  # Final model predictions for control pseudo-outcome
    
    # Average the two treatment effects to get the final treatment effect estimate
    treatment_effect_xlearner = 0.5 * (treatment_effect_treated + treatment_effect_control)
    
    return treatment_effect_xlearner, final_treated_model, final_control_model

# Train and evaluate the X-Learner with MSE calculations
x_treatment_effect, final_treated_model, final_control_model = x_learner(X_train_df, X_test_df, y_train, y_test, treatment_train, treatment_test)

# Print the estimated treatment effects
print("Estimated treatment effects (X-Learner):\n", x_treatment_effect[:5])  # Showing first 5 treatment effects


MSE for Treated Model (T-Learner): 0.47066167057112024
MSE for Control Model (T-Learner): 0.44565126147139134
MSE for Pseudo-Outcome Model (Treated): 0.22266443542350878
MSE for Pseudo-Outcome Model (Control): 0.25023735008863246
Estimated treatment effects (X-Learner):
 [0.11595774 0.43627006 0.13755668 1.15091238 0.7525621 ]


In [6]:
# Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Function to train and evaluate an R-Learner with MSE evaluation
def r_learner(X_train, X_test, y_train, y_test, treatment_train, treatment_test):
    # Ensure the indices of X_train, y_train, and treatment_train are aligned
    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    treatment_train = treatment_train.reset_index(drop=True)
    
    # Step 1: Estimate Propensity Scores (Logistic Regression for binary treatment)
    propensity_model = LogisticRegression(random_state=42)
    propensity_model.fit(X_train, treatment_train)
    propensity_scores = propensity_model.predict_proba(X_train)[:, 1]
    
    # Step 2: Estimate Outcome Models (treated and control)
    outcome_model_treated = RandomForestRegressor(n_estimators=100, random_state=42)
    outcome_model_control = RandomForestRegressor(n_estimators=100, random_state=42)
    
    # Train models on treated and control groups
    outcome_model_treated.fit(X_train[treatment_train == 1], y_train[treatment_train == 1])
    outcome_model_control.fit(X_train[treatment_train == 0], y_train[treatment_train == 0])
    
    # Predict outcomes for both treated and control cases
    y_pred_treated = outcome_model_treated.predict(X_train)
    y_pred_control = outcome_model_control.predict(X_train)
    
    # Step 2 MSE: Evaluate Outcome Models
    mse_treated = mean_squared_error(y_train[treatment_train == 1], y_pred_treated[treatment_train == 1])
    mse_control = mean_squared_error(y_train[treatment_train == 0], y_pred_control[treatment_train == 0])
    print(f"MSE for Treated Outcome Model: {mse_treated}")
    print(f"MSE for Control Outcome Model: {mse_control}")
    
    # Step 3: Calculate Residuals
    residuals_treated = (y_train - y_pred_treated) / propensity_scores
    residuals_control = (y_train - y_pred_control) / (1 - propensity_scores)
    
    # Step 4: Fit final treatment effect model (RandomForest)
    treatment_effect_model = RandomForestRegressor(n_estimators=100, random_state=42)
    treatment_effect_model.fit(X_train, treatment_train * residuals_treated + (1 - treatment_train) * residuals_control)
    
    # Predict treatment effects for the test set
    treatment_effect_test = treatment_effect_model.predict(X_test)
    
    # Step 4 MSE: Evaluate Treatment Effect Model
    mse_treatment_effect = mean_squared_error(y_test, treatment_effect_test)
    print(f"MSE for Treatment Effect Model: {mse_treatment_effect}")
    
    return treatment_effect_test, treatment_effect_model

# Ensure the treatment column for training and test sets has matching indices with features
treatment_train_aligned = treatment_train.reset_index(drop=True)
treatment_test_aligned = treatment_test.reset_index(drop=True)

# Train and evaluate the R-Learner with MSE calculations
r_treatment_effect, r_model = r_learner(X_train_df, X_test_df, y_train, y_test, treatment_train_aligned, treatment_test_aligned)

# Print the estimated treatment effects
print("Estimated treatment effects (R-Learner):\n", r_treatment_effect[:5])  # Showing first 5 treatment effects


MSE for Treated Outcome Model: 0.21964892203290864
MSE for Control Outcome Model: 0.24975125763615935
MSE for Treatment Effect Model: 0.45872479128868987
Estimated treatment effects (R-Learner):
 [ 0.01418077  0.0112741  -0.01358998 -0.12883522 -0.02388825]


In [7]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Fine-tuning the S-Learner using Gradient Boosting
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

# Use Gradient Boosting Regressor for the S-Learner
gbm = GradientBoostingRegressor(random_state=42)

# Handle missing values for grid search
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_df.join(treatment_train))

# Use GridSearchCV to find the best parameters for Gradient Boosting
grid_search = GridSearchCV(gbm, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_imputed, y_train)

# Get the best parameters and the tuned model
best_params = grid_search.best_params_
print("Best parameters for S-Learner (Gradient Boosting):", best_params)

# Pass the tuned model to the S-Learner
tuned_model = GradientBoostingRegressor(**best_params, random_state=42)
s_treatment_effect, s_model = s_learner(X_train_df, X_test_df, y_train, y_test, treatment_train, treatment_test, tuned_model)

# Print the estimated treatment effects
print("Estimated treatment effects (S-Learner, Tuned):\n", s_treatment_effect[:5])

Best parameters for S-Learner (Gradient Boosting): {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
S-Learner MSE: 0.3777395615180631
Estimated treatment effects (S-Learner, Tuned):
 [0. 0. 0. 0. 0.]


In [8]:
print("Treatment Distribution in Training Set:")
print(treatment_train.value_counts())

Treatment Distribution in Training Set:
Z
0    5601
1    2711
Name: count, dtype: int64


In [9]:
# Replace 'X1' with an actual feature name in your dataset
X_train_interaction = X_train.copy()
X_train_interaction['treatment_interaction'] = X_train['X1'] * treatment_train  # Interact X1 with treatment

X_test_interaction = X_test.copy()
X_test_interaction['treatment_interaction'] = X_test['X1'] * treatment_test  # Same for test set

# Use this new feature set for model training
s_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
s_treatment_effect, s_model = s_learner(X_train_interaction, X_test_interaction, y_train, y_test, treatment_train, treatment_test, s_model)

# Print the treatment effects
print("Estimated treatment effects (S-Learner, with interaction terms):\n", s_treatment_effect[:5])


S-Learner MSE: 0.2687659747632729
Estimated treatment effects (S-Learner, with interaction terms):
 [0. 0. 0. 0. 0.]


In [10]:
from sklearn.ensemble import RandomForestRegressor

# Use RandomForestRegressor instead of Gradient Boosting
s_model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
s_treatment_effect_rf, s_model_rf = s_learner(X_train_df, X_test_df, y_train, y_test, treatment_train, treatment_test, s_model_rf)

# Print the estimated treatment effects using Random Forest
print("Estimated treatment effects (S-Learner, Random Forest):\n", s_treatment_effect_rf[:5])


S-Learner MSE: 0.45278502287687317
Estimated treatment effects (S-Learner, Random Forest):
 [0. 0. 0. 0. 0.]


In [11]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

# Handle missing values by imputing them
imputer = SimpleImputer(strategy='mean')  # Replace missing values with the column mean
X_train_imputed = imputer.fit_transform(X_train_df.join(treatment_train))  # Impute training data

# Use Gradient Boosting Regressor as the base model for the S-learner (without GridSearch)
s_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Use cross-validation to check model performance across multiple folds
cv_scores = cross_val_score(s_model, X_train_imputed, y_train, cv=5, scoring='neg_mean_squared_error')

# Print the Cross-Validation MSE Scores
print("Cross-Validation MSE Scores:", -cv_scores)


Cross-Validation MSE Scores: [0.36535378 0.37363385 0.35033632 0.34084314 0.37415565]


In [12]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

# Step 1: Estimate propensity scores using Logistic Regression
propensity_model = LogisticRegression(random_state=42)
propensity_model.fit(X_train_df, treatment_train)

# Get propensity scores for training and test sets
propensity_scores_train = propensity_model.predict_proba(X_train_df)[:, 1]
propensity_scores_test = propensity_model.predict_proba(X_test_df)[:, 1]

# Step 2: Add propensity scores to the training and test datasets
X_train_with_ps = X_train_df.copy()
X_train_with_ps['propensity_score'] = propensity_scores_train

X_test_with_ps = X_test_df.copy()
X_test_with_ps['propensity_score'] = propensity_scores_test

# Step 3: Handle missing values by imputing them
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_with_ps)
X_test_imputed = imputer.transform(X_test_with_ps)

# Step 4: Create polynomial features
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_poly = poly.fit_transform(X_train_imputed)
X_test_poly = poly.transform(X_test_imputed)

# Step 5: Add treatment column to the imputed and polynomial features using np.column_stack
X_train_poly_with_treatment = np.column_stack((X_train_poly, treatment_train.values))  # Convert treatment to numpy array if needed
X_test_poly_with_treatment = np.column_stack((X_test_poly, treatment_test.values))  # Same for test set

# Step 6: Train the S-Learner using Gradient Boosting
s_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
s_model.fit(X_train_poly_with_treatment, y_train)

# Predict outcomes for both treated and untreated cases
X_test_poly_with_treatment[:, -1] = 1  # For treated cases
y_pred_treated = s_model.predict(X_test_poly_with_treatment)

X_test_poly_with_treatment[:, -1] = 0  # For untreated cases
y_pred_control = s_model.predict(X_test_poly_with_treatment)

# Calculate the treatment effect
treatment_effect = y_pred_treated - y_pred_control

# Step 7: Evaluate the model performance using mean squared error (MSE)
mse = mean_squared_error(y_test, s_model.predict(X_test_poly_with_treatment))
print(f"S-Learner MSE: {mse}")

# Print the estimated treatment effects
print("Estimated treatment effects (S-Learner, Polynomial Features):\n", treatment_effect[:5])


S-Learner MSE: 0.3864978186060779
Estimated treatment effects (S-Learner, Polynomial Features):
 [0.16798218 0.16823798 0.35765285 0.43156245 0.28123992]


1. Fine-Tune the Models Further
You’ve used models like Random Forests and Gradient Boosting. Consider:
Hyperparameter tuning (as you did with grid search).
Trying more advanced models (e.g., neural networks, causal forests) to see if they improve CATE estimation.
Exploring cross-validation further to check model performance stability across different data splits.
2. Deeper Analysis of Heterogeneity
Look at how CATE varies by different covariates. For example, does the treatment work better for certain subgroups (e.g., students with high prior achievement vs. low prior achievement)?
Use interaction terms in your models (e.g., interacting covariates with the treatment) to uncover deeper insights into how different factors influence the treatment effect.
3. Sensitivity Analysis
Test the robustness of your CATE estimates by performing sensitivity analyses. For instance, assess how the results change when you alter the covariates or use different sample sizes.
4. Policy Recommendations
Based on the results, you can make data-driven recommendations about the effectiveness of the treatment. If certain subgroups benefit more, it may inform targeted interventions or personalized treatments.
5. Explore Causal Inference Extensions
You could apply other causal inference methods, such as instrumental variables or causal forests, for further validation of your findings. These methods can offer different perspectives on estimating treatment effects.